## Elsevier API Documentation
https://dev.elsevier.com/documentation/ScienceDirectSearchAPI.wadl

### API Key Request

*   Navigate to the [Elsevier Developer Portal](https://dev.elsevier.com).
*   Sign in and request an API key. This generates a string that provides the value for the `els_apikey` variable in `credentials.py`.

In [2]:
import os
import csv
import requests
import pandas as pd
import re
import xmltodict
import json
from _credentials import keys

### Article filter
* Filtered for unique DOIs, restricted to Elsevier journal articles on the designated whitelist, and excluded review-like items (review/survey/overview).
* Applied a topic filter that excluded terms such as LCA and BIM unless “operation” or “control” appeared, followed by a hard filter for terms such as heat pump/exchanger, with triggers recorded.
* Executed a final inclusion filter retaining titles matching curated Model/Data/Application/Target keywords, and exported a strictly formatted file (DOI, Year, Title, Journal) with audit summaries.

In [ ]:
import os, re, csv, sys, json
import pandas as pd
from typing import List, Dict, Optional

# ======================= CONFIG =======================
OUT_ROOT = "csv_output"
RAW_DIR          = os.path.join(OUT_ROOT, "00_raw")
DEDUP_DIR        = os.path.join(OUT_ROOT, "01_dedup")
JDOI_DIR         = os.path.join(OUT_ROOT, "02_journal_only")
DESIG_DIR        = os.path.join(OUT_ROOT, "03_designated")
REVIEW_DIR       = os.path.join(OUT_ROOT, "04_reviews")
TOPIC_SOFT_DIR   = os.path.join(OUT_ROOT, "05_topic_soft")
TOPIC_HARD_DIR   = os.path.join(OUT_ROOT, "06_topic_hard")
KEYWORDS_DIR     = os.path.join(OUT_ROOT, "07_keywords")
FINAL_DIR        = os.path.join(OUT_ROOT, "99_final")

JOURNALS_LIST    = "csv/_journals.csv"
KEYWORDS_CSV     = "csv/_keywords-reduced.csv"

# Try these inputs in order
CANDIDATE_INPUTS = [
    os.path.join(OUT_ROOT, "fetching_1.csv"),
    os.path.join(OUT_ROOT, "elsevier_search_results_cleaned.csv"),
    "elsevier_search_results_cleaned.csv",
]

# Hard-exclusion terms (always drop; ignore allow pattern)
HARD_TERMS = [
    ("life cycle", r"\blife*cycle?\b"),
    ("district heating", r"\bdistrict\s*heat(ing)?\b"),
    ("district cooling", r"\bdistrict\s*cool(ing)?\b"),
    ("district",        r"\bdistrict(s)?\b"),
    ("stock",        r"\bstock(s)?\b"),
    ("city",        r"\bcity(s)?\b"),
    ("standard",        r"\bstandards\b"),
    ("grid",        r"\bgrid(s)?\b"),
    ("credit",        r"\bcredit(s)?\b"),
    ("dc",        r"\bdc(s)?\b"),
    ("ac",        r"\bac(s)?\b"),
    ("wave",        r"\bwave(s)?\b"),
    ("tide",        r"\btide(s)?\b"),
    ("tidal",        r"\btidal\b"),
    ("offshore",        r"\boff[-\s]?shore\b"),
    ("wind",        r"\bwind(s)?\b"),
    ("hydro",        r"\bhydro\b"),
    ("daylight",        r"\bdaylight(s)?\b"),
    ("lighting",        r"\blight(ing)?\b"),
    ("solar thermal",   r"\bsolar[-\s]?thermal\b"),
    ("microgrid", r"\bmicro[-\s]?grid(s)?\b"), # grid
    ("grid", r"\bgrid(s)?\b"), #
    ("embodied",        r"\bembodied?\b"),
    ("storage",        r"\bstorage(s)?\b"),
    ("material",            r"\bmaterial(s)?\b"),
    ("window",       r"\bwindow(s)?\b"),
    ("roof",         r"\broof(s)?\b"),
    ("ceiling",         r"\bceiling(s)?\b"),
    ("wall",         r"\bwall(s)?\b"),
    ("pcm",             r"\bpcm\b"),
    ("housing",       r"\bhousing?\b"),
    ("borehole",        r"\bborehole(s)?\b"),
    ("geothermal",      r"\bgeo[-\s]?thermal\b"),
    ("envelope",     r"\benvelope(s)?\b"),
    ("heat pump",        r"\bheat[-\s]?pump(s)?\b"),
    ("heat exchanger",   r"\bheat[-\s]?exchanger(s)?\b"),
    ("industry",         r"\bindustr(y|ies)\b"),
    ("glass",            r"\bglass(es)?\b"),
    ("glaze",            r"\bglaz(ing)?\b"),
    ("urban",            r"\burban?\b"),
    ("York",            r"\bYork?\b"),
    ("façade",            r"\bfaçade?\b"),
    ("ev",            r"\bev?\b"),
    ("electric vehicle",  r"\belectric[-\s]?vehicle(s)?\b"),
    ("mobility",         r"\bmobilit(y|ies)\b"),
    ("aircraft",        r"\baircraft(s)?\b"),
    ("transport",        r"\btransport(s|ation)?\b"),
    ("vehicle",          r"\bvehicle(s)?\b"),
    ("boiler",             r"\bboiler(s)\b"),
    ("chp",               r"\bchp\b"),
    ("cogeneration",      r"\bcogeneration\b"),
    ("trigeneration",     r"\btrigeneration\b"),
    ("cogen",            r"\bcogen\b"),
    ("trigen",           r"\btrigen\b"),
    ("fuel cell",        r"\bfuel[-\s]?cell(s)?\b"),
    ("combustion",       r"\bcombustion\b"),
    ("turbine",          r"\bturbine(s)?\b"),
    ("generator",        r"\bgenerator(s)?\b"),
    ("chiller",               r"\bchiller(s)?\b"),
    ("plant",                r"\bplant(s)\b"),
    ("tunnel",               r"\btunnel\b"),
    ("pv",               r"\bpv\b"),
    ("solar",            r"\bsolar\b"),
    ("indicators",            r"\bindicator(s)?\b"),
    ("evaluation",  r"\bevaluation\b"),
    ("construction", "construction"),
]

# Soft topic exclusion (unless allow_pattern hits)
SOFT_KEYWORDS = [
    ("roof", "roof(s)?"),
    ("wall", "wall(s)?"),
    ("envelope", "envelope(s)?"),
    ("design", "design(s)?"),
    ("compliance", "compliance"),
    ("retrofit", "retrofit(s|ting)?"),
    ("benchmark", "benchmark(ing|s)?"), 
    ("renovation", "renovation"),
    ("predict", "predict(ing)?"),
    ("forecast", "forecast"),
    ("lca", "lca"),
    ("life cycle assessment", r"life\s*cycle\s+assessment"),
    ("life cycle", r"life\s*cycle\s+"),
    ("emission", "emission(s)?"),
    ("embodied", "embodied"),
    ("sustainability", r"sustainab(le|ility)"),
    ("green", "green"),
    ("simulation", "simulation"),
    ("calibration", r"calibration"),
    ("roadmap", "roadmap"),
    ("annex", r"\bannex\b"),
    ("iea", r"\biea\b"),
    ("policy", "policy"),
    ("bim", "bim"),
    ("thermal comfort", r"thermal\s*comfort"),
    ("indoor air", r"indoor\s*air|indoor\s*air\s*quality|\biaq\b"),
]
ALLOW_PATTERN = re.compile(
    r"\b(?:operation|operations|operational|operate|operating|control|controls|controlled|controlling)\b",
    flags=re.IGNORECASE
)

REVIEW_LIKE = re.compile(
    r"\b(?:review|surveys?|overviews?|advance?|limit?|meta[-\s]?analysis)\b",
    flags=re.IGNORECASE
)


# ======================= UTILS ========================
def ensure_dirs(*paths: str) -> None:
    for p in paths:
        os.makedirs(p, exist_ok=True)

def write_csv(df: pd.DataFrame, path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=False)

def find_input() -> str:
    for p in CANDIDATE_INPUTS:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"No input CSV found in: {CANDIDATE_INPUTS}")

def load_df(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

def normalize_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    if "DOI" not in df.columns:
        dc = next((c for c in df.columns if c.lower() in ["dc:identifier","identifier"]), None)
        if not dc: raise ValueError("No DOI column (expected 'DOI' or 'dc:identifier').")
        df["DOI"] = df[dc].astype(str).str.replace("DOI:", "", regex=False).str.strip()
    title_col = next((c for c in ["Title","dc:title","title"] if c in df.columns), None)
    journal_col = next((c for c in ["Journal","prism:publicationName","journal","publicationName"] if c in df.columns), None)
    if not journal_col:
        raise ValueError("No journal column (e.g., 'Journal' or 'prism:publicationName').")
    return {"title_col": title_col, "journal_col": journal_col}

def _norm_name(s) -> str:
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("&", "and")
    return s

def load_journal_whitelist(path: str) -> set:
    jl = pd.read_csv(path)
    possible = ["Journal","journal","Title","title","publication","Publication"]
    col = next((c for c in possible if c in jl.columns), jl.columns[0])
    return set(jl[col].dropna().map(_norm_name).drop_duplicates().tolist())

def is_review_flag(df: pd.DataFrame) -> pd.Series:
    cols_lower = {c.lower(): c for c in df.columns}
    review = pd.Series(False, index=df.index)
    if "subtype" in cols_lower:
        st_col = cols_lower["subtype"]
        st = df[st_col].astype(str).str.lower()
        review |= st.eq("re") | df[st_col].astype(str).str.contains(REVIEW_LIKE, na=False)
    if "subtypedescription" in cols_lower:
        sd_col = cols_lower["subtypedescription"]
        review |= df[sd_col].astype(str).str.contains(REVIEW_LIKE, na=False)
    title_col = cols_lower.get("title", cols_lower.get("dc:title"))
    if title_col is not None:
        undecided = ~review
        if undecided.any():
            t = df.loc[undecided, title_col].astype(str)
            review.loc[undecided] = t.str.contains(REVIEW_LIKE, na=False)
    return review

def soft_topic_flags(titles: pd.Series):
    matched_lists = []
    for name, pat in SOFT_KEYWORDS:
        hit = titles.str.contains(pat, flags=re.IGNORECASE, na=False)
        matched_lists.append(hit.map(lambda x: name if x else ""))
    reasons = pd.Series([",".join(filter(None, row)) for row in zip(*matched_lists)], index=titles.index)
    has_kw = reasons.astype(bool)
    has_allow = titles.str.contains(ALLOW_PATTERN, na=False)
    topic_exclude = has_kw & (~has_allow)
    return topic_exclude, reasons

def hard_topic_flags(titles: pd.Series):
    matched_lists = []
    for name, pat in HARD_TERMS:
        hit = titles.str.contains(pat, flags=re.IGNORECASE, na=False)
        matched_lists.append(hit.map(lambda x: name if x else ""))
    reasons = pd.Series([",".join(filter(None, row)) for row in zip(*matched_lists)], index=titles.index)
    return reasons.astype(bool), reasons

def phrase_to_regex(phrase: str):
    tokens = re.split(r"[-\s]+", phrase.strip())
    tokens = [t for t in tokens if t]
    if not tokens: return None
    parts = [re.escape(tok) + r"\w*" for tok in tokens]  # allow plurals/affixes & embedded
    return re.compile(r"(?:%s)" % r"[-\s]*".join(parts), flags=re.IGNORECASE)

def split_terms(s: str) -> List[str]:
    if pd.isna(s): return []
    return [x.strip() for x in re.split(r"[;,]", str(s)) if x and x.strip()]

def keyword_inclusion(df: pd.DataFrame, title_col: Optional[str]):
    kw_df = pd.read_csv(KEYWORDS_CSV)
    cmap = {c.lower(): c for c in kw_df.columns}
    cat_col = cmap.get("category"); keys_col = cmap.get("keywords"); ref_col = cmap.get("ref")
    if not cat_col or not keys_col:
        raise ValueError("`csv/_keywords.csv` must have 'Category' and 'Keywords'.")

    compiled = []
    for _, row in kw_df.iterrows():
        cat = str(row[cat_col]).strip()
        base_terms = split_terms(row[keys_col])
        ref_terms = split_terms(row[ref_col]) if (ref_col and pd.notna(row[ref_col])) else []
        for term in base_terms + ref_terms:
            rx = phrase_to_regex(term)
            if rx is not None:
                compiled.append({"category": cat, "term": term, "regex": rx})

    if not title_col or not compiled:
        df["_include_terms"] = ""; df["_include_categories"] = ""; df["_include_any"] = False
        counts_df = pd.DataFrame(columns=["Category","Keyword","Match_Count"])
        audit_df = df[[]]
        return df, audit_df, counts_df

    t = df[title_col].astype(str)
    term_names = [it["term"] for it in compiled]
    term_cats  = [it["category"] for it in compiled]
    term_series = [t.str.contains(it["regex"], na=False) for it in compiled]

    matched_term_names = []
    matched_categories = []
    for row_bools in zip(*term_series):
        names = [n for n, hit in zip(term_names, row_bools) if hit]
        cats  = {c for c, hit in zip(term_cats,  row_bools) if hit}
        matched_term_names.append(",".join(names))
        matched_categories.append(",".join(sorted(cats)))

    df["_include_terms"] = pd.Series(matched_term_names, index=df.index)
    df["_include_categories"] = pd.Series(matched_categories, index=df.index)
    df["_include_any"] = df["_include_terms"].astype(bool)

    counts = [{"Category": it["category"], "Keyword": it["term"], "Match_Count": int(s.sum())}
              for it, s in zip(compiled, term_series)]
    counts_df = pd.DataFrame(counts).groupby(["Category","Keyword"], as_index=False)["Match_Count"].sum()

    audit_cols = ["DOI", "Year"] + ([title_col] if title_col else []) + ["_include_terms","_include_categories"]
    audit_df = df[audit_cols].copy()
    return df, audit_df, counts_df

def tidy(df: pd.DataFrame, title_col: Optional[str], journal_col: str) -> pd.DataFrame:
    cols = ["DOI", "Year"] + ([title_col] if title_col else []) + [journal_col]
    exist = [c for c in cols if c in df.columns]
    rest = [c for c in df.columns if c not in exist and not c.startswith("_")]
    return df[exist + rest]

def save_final_exact(df: pd.DataFrame, title_col: Optional[str], journal_col: str, path: str):
    out = df.copy()
    if title_col and "Title" not in out.columns:
        out["Title"] = out[title_col]
    if "Journal" not in out.columns:
        out["Journal"] = out[journal_col]
    out = out[["DOI","Year","Title","Journal"]]
    write_csv(out, path)
    return out

def stage_summary_line(name: str, kept: int, excl: Optional[int] = None) -> str:
    return f"{name}: {kept}" + (f"\n(Excluded: {excl})" if excl is not None else "")

def contains_ci(s: pd.Series, pat):
    if isinstance(pat, re.Pattern):          # already compiled with flags
        return s.str.contains(pat, na=False, regex=True)
    return s.str.contains(pat, na=False, regex=True, case=False)

# ===================== STAGE-ONLY RUN =====================
def run_pipeline_stage_only():
    ensure_dirs(RAW_DIR, DEDUP_DIR, JDOI_DIR, DESIG_DIR, REVIEW_DIR, TOPIC_SOFT_DIR, TOPIC_HARD_DIR, KEYWORDS_DIR, FINAL_DIR)

    input_csv = find_input()
    df = load_df(input_csv)
    mapping = normalize_columns(df)
    title_col, journal_col = mapping["title_col"], mapping["journal_col"]

    write_csv(df, os.path.join(RAW_DIR, os.path.basename(input_csv)))
    print("Original:", len(df))

    # 1) De-dup DOI
    df = df.dropna(subset=["DOI"]).drop_duplicates(subset=["DOI"])
    write_csv(tidy(df, title_col, journal_col), os.path.join(DEDUP_DIR, "dedup.csv"))
    print(stage_summary_line("Deduplication", len(df)))

    # 2) Journal DOIs only (/j.)
    is_journal = df["DOI"].astype(str).str.contains(r"/j\.", case=False, na=False)
    books_df = tidy(df.loc[~is_journal].copy(), title_col, journal_col)
    df = df.loc[is_journal].copy()
    write_csv(books_df, os.path.join(JDOI_DIR, "excluded_books.csv"))
    write_csv(tidy(df, title_col, journal_col), os.path.join(JDOI_DIR, "journal_only.csv"))
    print(stage_summary_line("* After filtering books/conference papers", len(df), excl=len(books_df)))

    # 3) Designated journals
    whitelist = load_journal_whitelist(JOURNALS_LIST)
    df["_journal_norm"] = df[journal_col].map(_norm_name)
    in_designated = df["_journal_norm"].isin(whitelist)
    not_designated_df = tidy(df.loc[~in_designated].copy(), title_col, journal_col)
    df = df.loc[in_designated].copy()
    write_csv(not_designated_df, os.path.join(DESIG_DIR, "excluded_not_designated.csv"))
    write_csv(tidy(df, title_col, journal_col), os.path.join(DESIG_DIR, "designated.csv"))
    print(stage_summary_line("* After filtering those are not published in designated journals", len(df), excl=len(not_designated_df)))

    # 4) Remove reviews (review/survey/overview/etc.)
    df["_is_review"] = is_review_flag(df)  # keep your function but ensure it uses non-capturing groups internally

    reviews_df = tidy(df.loc[df["_is_review"]].copy(), title_col, journal_col)
    df = df.loc[~df["_is_review"]].copy()

    write_csv(reviews_df, os.path.join(REVIEW_DIR, "excluded_reviews.csv"))
    write_csv(tidy(df, title_col, journal_col), os.path.join(REVIEW_DIR, "no_reviews.csv"))
    print(stage_summary_line("* After removing reviews", len(df), excl=len(reviews_df)))

    # 5) Soft topic exclusion (unless allow pattern)
    if title_col:
        titles = df[title_col].astype(str).str.lower()
        topic_excl, reasons = soft_topic_flags(titles)
        df["_topic_exclude"] = topic_excl
        df["_exclude_reason"] = reasons
    else:
        df["_topic_exclude"] = False
        df["_exclude_reason"] = ""
    topic_excl_df = tidy(df.loc[df["_topic_exclude"]].copy(), title_col, journal_col)
    df = df.loc[~df["_topic_exclude"]].copy()
    write_csv(topic_excl_df, os.path.join(TOPIC_SOFT_DIR, "excluded_topic_keywords.csv"))
    write_csv(tidy(df, title_col, journal_col), os.path.join(TOPIC_SOFT_DIR, "passed_soft_topic.csv"))
    print(stage_summary_line("* After soft topic exclusion", len(df), excl=len(topic_excl_df)))

    # 6) Hard topic exclusion (always drop)
    if title_col:
        titles = df[title_col].astype(str).str.lower()
        hard_mask, hard_reasons = hard_topic_flags(titles)
        df["_hard_topic_exclude"] = hard_mask
        df["_hard_exclude_reason"] = hard_reasons
    else:
        df["_hard_topic_exclude"] = False
        df["_hard_exclude_reason"] = ""
    hard_df = tidy(df.loc[df["_hard_topic_exclude"]].copy(), title_col, journal_col)
    df = df.loc[~df["_hard_topic_exclude"]].copy()
    write_csv(hard_df, os.path.join(TOPIC_HARD_DIR, "excluded_hard_topic_keywords.csv"))
    write_csv(tidy(df, title_col, journal_col), os.path.join(TOPIC_HARD_DIR, "passed_hard_topic.csv"))
    print(stage_summary_line("* After hard topic exclusion", len(df), excl=len(hard_df)))

    # 7) Keyword-driven inclusion (keeps only titles matching keywords from csv/_keywords.csv)
    df, audit_df, counts_df = keyword_inclusion(df, title_col)
    df = df.loc[df["_include_any"]].copy()
    audit_df.to_csv(os.path.join(KEYWORDS_DIR, "included_keyword_matches.csv"), index=False)
    counts_df.to_csv(os.path.join(KEYWORDS_DIR, "keyword_match_counts.csv"), index=False)
    print(stage_summary_line("* After keyword inclusion", len(df)))

    # FINAL (exact columns only)
    final_path = os.path.join(FINAL_DIR, "elsevier_final_clean.csv")
    final_out = save_final_exact(df, title_col, journal_col, final_path)
    print(f"...Final Saved... \n{final_path}")

if __name__ == "__main__":
    run_pipeline_stage_only()

Original: 48100
Deduplication: 47106
* After filtering books/conference papers: 45342
(Excluded: 1764)
* After filtering those are not published in designated journals: 23247
(Excluded: 22095)
* After removing reviews: 21816
(Excluded: 1431)


C:\Users\s2589602\AppData\Local\Temp\ipykernel_17580\2184806677.py:200: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit = titles.str.contains(pat, flags=re.IGNORECASE, na=False)


* After soft topic exclusion: 13881
(Excluded: 7935)


C:\Users\s2589602\AppData\Local\Temp\ipykernel_17580\2184806677.py:211: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit = titles.str.contains(pat, flags=re.IGNORECASE, na=False)


* After hard topic exclusion: 7749
(Excluded: 6132)
* After keyword inclusion: 4177
...Final Saved... 
csv_output\99_final\elsevier_final_clean.csv
